In [ ]:
import re

def normalize(text):
    return re.sub(r'\W+', ' ', text.lower().strip())

def iou(prediction, ground_truth):
    tokenized_prediction = set(normalize(prediction).split())
    tokenized_ground_truth = set(normalize(ground_truth).split())
    return len(tokenized_prediction.intersection(tokenized_ground_truth)) / len(tokenized_ground_truth)
from fuzzywuzzy import fuzz
def simplify_cat(cat):
    return re.sub(r'\s*\(.*?\)', '', cat) 
# Transformed categories
# categories = simplify_categories(categories)
def get_match(prediction, categories):
    prediction = simplify_cat(prediction)
    best_match = max(categories, key=lambda cat: fuzz.partial_ratio(prediction, cat))
    if fuzz.partial_ratio(prediction, best_match) > 70:  # Example threshold
        return best_match
    return "Unknown"

categories=[simplify_cat(cat) for cat in ['Realistic', 'Non-realistic (e.g. cartoon, illustration, graphic).', 'Unknown']]
# categories=[simplify(cat) for cat in ['Realistic', 'Non-realistic', 'Unknown']]
print(categories)
predictions=['[2] Non-realistic (e.g. cartoon, illustration, graphic)', '[2] Non-realistic', '[2] Non-realistic', '[2] Non-realistic', '[2] Non-realistic', '[2] Non-realistic', '[1] Realistic', '[2] Non-realistic', '[1] Realistic', '[2] Non-realistic', '[2] Non-realistic', '[1] Realistic', '[2] Non-realistic', '[2] Non-realistic', '[1] Realistic', '[1] Realistic', '[2] Non-realistic', '[2] Non-realistic', '[2] Non-realistic', '[2] Non-realistic']
attribute_classifications = [get_match(pred, categories) for pred in predictions]
# attribute classifications output
# ['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Realistic', 'Unknown', 'Realistic', 'Unknown', 'Unknown', 'Realistic', 'Unknown', 'Unknown', 'Realistic', 'Realistic', 'Unknown', 'Unknown', 'Unknown', 'Unknown']
# best_match = max(categories, key=lambda cat: fuzz.partial_ratio(predictions[1], cat))
# best_match
attribute_classifications

In [2]:
class Context:
    def __init__(self):
        self.context_strings = []
        self.actions = []
        self.action_assoc_with_context = []
        
    def add(self, string):
        if isinstance(string, list):
            string = str(string)
        self.context_strings.append(string)
        if len(self.actions) > 0:
            text = f'Step {len(self.actions)} [{self.actions[-1]}]: '
        else:
            text = f'Step {len(self.actions)} [Initial]: '


        self.action_assoc_with_context.append(text)
        
    def step(self, action):
        self.actions.append(action)
        
    def get_num_steps(self):
        return len(self.actions)
        
    def get_conclusion(self):
        if "conclusion" in self.actions[-1]:
            return self.context_strings[-1]
        else:
            return None
    def read(self):
        return '\n'.join(self.context_strings)
    def read_presentation(self):
        context_string_with_prefix = []
        for prefix, context in zip(self.action_assoc_with_context, self.context_strings):
            context_string_with_prefix.append(prefix+context)
        return '\n'.join(context_string_with_prefix)
        
    def read_exp(self):
        # read last three items
        return '\n'.join(self.context_strings[-4:])

In [3]:
import pickle

# Load the pickle file
with open("eval_outputs/saved_outputn=202.pkl", "rb") as file:
    data = pickle.load(file)

# Access and print the desired variables
errors = data["errors"]
conclusions = data["conclusions"]

print("Errors:", errors)
# print("Conclusions:", conclusions)


Errors: [Exception('Invalid Slice Condition'), Exception('Invalid Slice Condition'), Exception('Invalid Slice Condition'), KeyError('obstruction'), Exception('Invalid Slice Condition'), Exception('Invalid Slice Condition'), Exception('Invalid Slice Condition'), Exception('Invalid Slice Condition'), Exception('Invalid Slice Condition')]


In [5]:
data['gt_successful'][0]

('art', 'saxophone')

In [6]:
object_mention_acc = 0
attribute_mention_acc = 0
total_valid_conclusion = 0
successful_examples = []
incorrect_examples = []
correct_attribute_examples = []

for i, conclusion in enumerate(conclusions):
    if conclusion is None:
        continue
    attribute, object_name = data['gt_successful'][i]
    
    # Check if the attribute and object name are mentioned in the conclusion
    object_mentioned = object_name in conclusion
    attribute_mentioned = attribute in conclusion
    
    # Update counters
    if object_mentioned and attribute_mentioned:
        successful_examples.append((i, conclusion))
    else:
        incorrect_examples.append((i, conclusion))
    
    if attribute_mentioned:
        correct_attribute_examples.append((i, conclusion))
    
    object_mention_acc += int(object_mentioned)
    attribute_mention_acc += int(attribute_mentioned)
    total_valid_conclusion += 1

# Calculate final accuracies
object_mention_acc = object_mention_acc / total_valid_conclusion if total_valid_conclusion > 0 else 0
attribute_mention_acc = attribute_mention_acc / total_valid_conclusion if total_valid_conclusion > 0 else 0

# Output results
print(f"Object Mention Accuracy: {object_mention_acc:.2f}")
print(f"Attribute Mention Accuracy: {attribute_mention_acc:.2f}")
print(f"Total Valid Conclusions: {total_valid_conclusion}")
print(f"Both Correct Examples: {len(successful_examples)/total_valid_conclusion:.2f}")
# print(f"Incorrect Examples: {incorrect_examples}")
# print(f"Correct Attribute Examples: {correct_attribute_examples}")


Object Mention Accuracy: 0.59
Attribute Mention Accuracy: 0.07
Total Valid Conclusions: 95
Both Correct Examples: 0.04


In [22]:
for i, conclusion in incorrect_examples:
    print( data['gt_successful'][i])
    print(conclusion)
    print("----------")

('sculpture', 'scuba_diver')
Since the experiment has been conducted and the results show that the model is struggling with classes that have a high degree of visual similarity, ambiguity, or variability in their representations, the next logical step is to draw conclusions based on the observations and error rates. 

The action should be to 'conclusion' to summarize the attribute and object class that are contributing to the model's errors. 

Conclusion: The model is struggling with classes that have a high degree of visual similarity, ambiguity, or variability in their representations, which makes it difficult for the model to accurately distinguish them from other classes. The object class 'scuba_diver' is one of the top five classes with the highest errors, and its images have visual features that are similar to other classes, such as 'beer_glass', which are contributing to the model's errors.
----------
('misc', 'dragonfly')
The context shows that we have detailed descriptions or 

In [7]:
def find_match(string):
    for context in data["contexts"]:
        if string in context.read():
            return context
    return None

In [23]:
matched_context = find_match("Since the experiment has been conducted, the next step is to draw conclusions based on the results. The error rates indicate that the majority of the error is correlated with the 'Unknown' category, suggesting that the model is struggling to distinguish 'hummingbird' from other objects. Therefore, the action should be to 'conclusion' and summarize the attribute and object class.")

In [24]:
matched_context.actions

['experiment', 'conclusion']

In [26]:
print(matched_context.read_presentation())

Step 0 [Initial]: The top five classes with the highest errors are: [('hummingbird', np.float64(0.08959537572254335)), ('newt', np.float64(0.05128205128205128)), ('assault_rifle', np.float64(0.043010752688172046)), ('junco', np.float64(0.03571428571428571)), ('french_bulldog', np.float64(0.03488372093023256))]
Step 0 [Initial]: Since we already have detailed descriptions or observations for some object classes, the next step would be to conduct an experiment. Let's focus on the class 'hummingbird', which has the highest error rate. We can formulate a hypothesis based on the context and observations, and then test it through experimentation. 

Action: experiment
Step 1 [experiment]: The model is likely misclassifying images of 'hummingbird' because it is having trouble distinguishing the object from other small, fast-moving objects or objects with similar visual characteristics, such as 'newt' or 'junco', which are also small birds.
Step 1 [experiment]: For each image, label from three 